<a href="https://colab.research.google.com/github/profteachkids/chetools/blob/main/tools/ipopt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:36
🔁 Restarting kernel...


In [2]:
!conda install -c conda-forge cyipopt

Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - cyipopt


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ampl-mp-3.1.0              |    h2cc385e_1006         1.1 MB  conda-forge
    certifi-2022.6.15          |     pyhd8ed1ab_1         154 KB  conda-forge
    cyipopt-1.1.0              |   py37hb2fd402_3         135 KB  conda-forge
    future-0.18.2              |   py37h89c1867_5         713 KB  conda-forge
    ipopt-3.14.8               |       hc8a599a_0         1.1 MB  conda-forge
    libblas-3.9.0              |16_linux64_openblas          13 KB  conda-forge
    libcblas-3.9.0             |16_linux64_openblas          13 KB  conda-forge
  

In [1]:
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

import cyipopt
from cyipopt import minimize_ipopt
from functools import partial

In [15]:
class Problem():
    def __init__(self,obj,constraints,x0,show_intermediate=True):
        self.x0=x0
        self.show_intermediate=show_intermediate
        self.obj=obj
        self.obj_jit=jax.jit(obj)
        self.obj_grad = jax.jit(jax.grad(obj))
        self.constraints_functions=constraints
        self.constraint_jac = jax.jit(jax.jacobian(self.combined_constraints))
        self.hess = jax.jit(jax.jacrev(jax.jacfwd(self.L)))
        self.n_constraints = self.combined_constraints(self.x0).size
        self.row, self.col = self.hessianstructure()


    def objective(self, x):
        return self.obj_jit(x)

    def gradient(self, x):
        return self.obj_grad(x)

    @partial(jax.jit, static_argnums=(0,))
    def combined_constraints(self, x):
        constraint_list=[jnp.atleast_1d(f(x)) for f in self.constraints_functions]
        if len(constraint_list) == 0:
            return jnp.array([])
        return jnp.concatenate(constraint_list)

    def constraints(self, x):
        return self.combined_constraints(x)

    def jacobian(self, x):
        return self.constraint_jac(x)



    @partial(jax.jit, static_argnums=(0,))
    def L(self, x, lagrange, obj_factor):
        return obj_factor*self.obj(x) + jnp.sum(lagrange*self.combined_constraints(x))

    def hessian(self, x, lagrange, obj_factor):
        return self.hess(x,lagrange,obj_factor)[self.row, self.col]

    def hessianstructure(self):
        return jnp.nonzero(jnp.tril(self.hess(self.x0,jnp.ones(self.n_constraints), 1.)))

    def intermediate(self, alg_mod, iter_count, obj_value, inf_pr, inf_du, mu,
                     d_norm, regularization_size, alpha_du, alpha_pr,
                     ls_trials):
        """Prints information at every Ipopt iteration."""
        if not self.show_intermediate:
            return
        msg = "Objective value at iteration #{:d} is - {:g}"

        print(msg.format(iter_count, obj_value))



In [18]:
# n_grades = 100
# grades = jnp.arange(n_grades)


# def entropy(p):
#     return jnp.sum(p[:n_grades]*jnp.log(p[:n_grades]))

# def p_constraint(p):
#     return jnp.sum(p[:n_grades])-1.

# def avg_constraint(p):
#     return jnp.sum(p[:n_grades]*grades)-50.

# def std_constraint(p):
#     avg = jnp.sum(p[:n_grades]*grades)
#     std= jnp.sqrt(jnp.sum(p[:n_grades]*(grades - avg)**2 )) - 10.
#     return std

# problem=Problem(entropy, [entropy, p_constraint, avg_constraint, std_constraint], jnp.full(n_grades,1/n_grades), show_intermediate=True)
# x0 = jnp.full(n_grades,1/n_grades)
# nlp=cyipopt.Problem(
#    n=len(x0),
#    m=problem.n_constraints,
#    problem_obj=problem,
#    lb=[0.]*len(x0),
#    ub=[1.]*len(x0),
#    cl=[0.]*problem.n_constraints,
#    cu=[0.]*problem.n_constraints,
# )
# res=nlp.solve(x0)
# res[0]